In [1]:
# stcr_validator.py
import json, os, math, re

BASE = r"C:\Users\lenovo\Desktop\final engine\data"   # change if needed
PATH_MAIN = os.path.join(BASE, "production_dataset_final_ready.json")
PATH_STCR_LIB = os.path.join(BASE, "stcr_equation_constants.json.json")

def load(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def find_vars(expr):
    return set(re.findall(r'\b(T|SN|SP|SK|ON|OP|OK)\b', expr or ""))

def safe_eval(expr, vars_):
    try:
        return eval(expr, {"__builtins__": None}, vars_)
    except Exception:
        return None

def check_entry(state,crop,block):
    eqs = block.get("equations", {})
    # quick numeric tests
    tests = [
        {"T":1,"SN":50,"SP":20,"SK":100,"ON":0,"OP":0,"OK":0},
        {"T":40,"SN":140,"SP":20,"SK":140,"ON":0,"OP":0,"OK":0},
        {"T":60,"SN":80,"SP":10,"SK":80,"ON":0,"OP":0,"OK":0},
    ]
    issues=[]
    for nutrient in ["N","P2O5","K2O"]:
        expr = eqs.get(nutrient)
        if not expr:
            issues.append(f"missing equation for {nutrient}")
            continue
        # detect variables used
        vars_used = find_vars(expr)
        # detect suspicious coefficients: look for *SP or *SK multipliers with large magnitude
        for m in re.finditer(r'([+-]?\d+(\.\d+)?)\s*\*\s*(SN|SP|SK)', expr):
            coeff = float(m.group(1))
            var = m.group(3)
            if abs(coeff) > 2.0:
                issues.append(f"large coeff {coeff} on {var} in {nutrient}")
        # evaluate on tests
        vals = []
        for t in tests:
            v = safe_eval(expr, t)
            if v is None:
                issues.append(f"eval error for {nutrient}: '{expr}'")
                v = 0
            vals.append(v)
        # flag if typically negative for reasonable T
        neg_count = sum(1 for v in vals if (v is not None and v < -10))
        if neg_count >= 2:
            issues.append(f"{nutrient} returns highly negative values for many testcases: {vals}")
    return issues

def run():
    main = load(PATH_MAIN)
    stcr_lib = None
    try:
        stcr_lib = load(PATH_STCR_LIB)
    except:
        pass

    report = {}
    for state,crops in main["data"].items():
        for crop,entry in crops.items():
            stcr_list = entry.get("stcr", [])
            for i,b in enumerate(stcr_list):
                issues = check_entry(state,crop,b)
                if issues:
                    report.setdefault(state, {}).setdefault(crop, []).append({"block_index":i,"soil_type":b.get("Soil_Type"), "issues":issues, "equations": b.get("equations")})
    print("==== STCR VALIDATION REPORT ====")
    if not report:
        print("No obvious issues found by quick checks.")
    else:
        import pprint; pprint.pprint(report)

if __name__ == "__main__":
    run()


==== STCR VALIDATION REPORT ====
{'Andhra Pradesh': {'Brinjal': [{'block_index': 0,
                                 'equations': {'K2O': '0.65*T - 0.34*SK - '
                                                      '0.52*OK',
                                               'N': '0.69*T - 0.72*SN - '
                                                    '0.64*ON',
                                               'P2O5': '0.41*T - 3.57*SP - '
                                                       '0.72*OP'},
                                 'issues': ['N returns highly negative values '
                                            'for many testcases: [-35.31, '
                                            '-73.2, -16.199999999999996]',
                                            'large coeff 3.57 on SP in P2O5',
                                            'P2O5 returns highly negative '
                                            'values for many testcases: '
                                   